In [18]:
import pandas as pd
import argilla as rg

data_file = "data/data_gn.pkl"

dataset = pd.read_pickle(data_file)



In [19]:
def to_conll_eval_format(data):
    output = []
    for d in data:
        sentence_tags = []
        for token in d['tokens']:
            sentence_tags.append(token['tag'])
        output.append(sentence_tags)
    return output

tokens = dataset["tokens"].tolist()
conll = to_conll_eval_format(dataset["metrics"].tolist())



In [20]:
conll_total = [c for s in conll for c in s]
set(conll_total)

{'B-NG', 'I-NG', None, 'O'}

In [24]:
import pandas as pd
data = pd.DataFrame({"tokens": tokens, "ng_tags": conll})
data = data.reset_index()
# Rename index to id
data = data.rename(columns={"index": "id"})
#data = data.to_dict('records')


In [32]:
data

,id,tokens_list,ng_tags
0,0,"[['@lobitachilena', '@camilapfer', 'Delincuent...","['B-NG', 'B-NG', 'B-NG', 'O', 'O', 'B-NG', 'O'..."
1,1,"[['La', 'frase', 'de', 'la', 'noche', '""', 'Er...","['B-NG', 'I-NG', 'I-NG', 'I-NG', 'I-NG', 'O', ..."
2,2,"[['Q', 'May', 'tu', 'comentario', ',', 'yo@no'...","['O', 'B-NG', 'B-NG', 'I-NG', 'O', 'O', 'O', '..."
3,3,"[['@PDI_Araucania', '@clintjoselint', 'Es', 'u...","['B-NG', 'B-NG', 'O', 'B-NG', 'I-NG', 'I-NG', ..."
4,4,"[['@carodream1', 'Hoy', 'ha', 'sido', 'un', 'd...","['B-NG', 'O', 'O', 'O', 'B-NG', 'I-NG', 'I-NG'..."
...,...,...,...
2608,2608,"[['Deberían', 'haber', 'dos', 'twiter', 'uno',...","['O', 'O', 'B-NG', 'I-NG', 'B-NG', 'I-NG', 'I-..."
2609,2609,"[['@VeronicG', '_', '_', '@sebastianpinera', '...","['B-NG', 'I-NG', 'I-NG', 'B-NG', 'B-NG', 'O', ..."
2610,2610,"[['@azetaene', 'Y', 'sigue', 'pegado', 'en', '...","['B-NG', 'O', 'O', 'O', 'O', 'B-NG', 'I-NG', '..."
2611,2611,"[['@ignaciobriones', '_', 'Briones', 'deja', '...","['B-NG', 'I-NG', 'B-NG', 'O', 'O', 'O', 'O', '..."


In [29]:
from datasets import Dataset, Features, ClassLabel, Sequence

features = Features({"ng_tags": Sequence(ClassLabel(num_classes=3, names=['B-NG', 'I-NG', 'O']))})

#hf_dataset = Dataset.from_list(data, features=features)

hf_dataset = Dataset.from_pandas(data, features=features)

hf_dataset.features

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_3719785/3274544608.py:7 in <module>                                               │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_3719785/3274544608.py'                      │
│                                                                                                  │
│ /home/jlortiz/data/anaconda3/envs/py310/lib/python3.10/site-packages/datasets/arrow_dataset.py:8 │
│ 11 in from_pandas                                                                                │
│                                                                                                  │
│    808 │   │   if features is not None:                                                          │
│    809 │   │   │   # more expensive cast than InMemoryTable.from_pandas(..., schema=features.ar  │
│    810 │   │   │   # needed to support the str to Audio conversion for instance                  │
│ ❱  811 │   │   │   table = table.cast(features.arrow_schema)                                     │
│    812 │   │   return cls(table, info=info, split=split)                                         │
│    813 │                                                                                         │
│    814 │   @classmethod                                                                          │
│                                                                                                  │
│ /home/jlortiz/data/anaconda3/envs/py310/lib/python3.10/site-packages/datasets/table.py:907 in    │
│ cast                                                                                             │
│                                                                                                  │
│    904 │   │   Returns:                                                                          │
│    905 │   │   │   `datasets.table.Table`                                                        │
│    906 │   │   """                                                                               │
│ ❱  907 │   │   return InMemoryTable(table_cast(self.table, *args, **kwargs))                     │
│    908 │                                                                                         │
│    909 │   def replace_schema_metadata(self, *args, **kwargs):                                   │
│    910 │   │   """                                                                               │
│                                                                                                  │
│ /home/jlortiz/data/anaconda3/envs/py310/lib/python3.10/site-packages/datasets/table.py:2312 in   │
│ table_cast                                                                                       │
│                                                                                                  │
│   2309 │   │   table (`pyarrow.Table`): the casted table                                         │
│   2310 │   """                                                                                   │
│   2311 │   if table.schema != schema:                                                            │
│ ❱ 2312 │   │   return cast_table_to_schema(table, schema)                                        │
│   2313 │   elif table.schema.metadata != schema.metadata:                                        │
│   2314 │   │   return table.replace_schema_metadata(schema.metadata)                             │
│   2315 │   else:                                                                                 │
│                                                                                                  │
│ /home/jlortiz/data/anaconda3/envs/py310/lib/python3.10/site-packages/datasets/table.py:2270 in   │
│ cast_table_to_schema                                       

In [ ]:
from datasets import Dataset, Features, ClassLabel
labels = [0, 0, 1]
features = Features({"label": ClassLabel(names=["negative", "positive"])})
ds = Dataset.from_dict({"label": labels}, features=features) 
ds = ds.with_format("tf")  
ds[:3]

In [ ]:
hf_dataset = hf_dataset.train_test_split(test_size=0.2, shuffle=True, seed=13)
# Rename datasets["test"] to datasets["validation"]
hf_dataset["validation"] = hf_dataset["test"]
del hf_dataset["test"]
hf_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ng_tags'],
        num_rows: 2090
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ng_tags'],
        num_rows: 523
    })
})

In [ ]:
hf_dataset.push_to_hub("spanish_nominal_groups_conll2003")

[05/31/23 14:00:41] WARNING  WARNING:datasets.dataset_dict:Pushing split train to the Hub.     ]8;id=291077;file:///home/jlortiz/data/anaconda3/envs/py310/lib/python3.10/site-packages/datasets/dataset_dict.py\dataset_dict.py]8;;\:]8;id=44341;file:///home/jlortiz/data/anaconda3/envs/py310/lib/python3.10/site-packages/datasets/dataset_dict.py#1532\1532]8;;\

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

[05/31/23 14:00:45] WARNING  WARNING:datasets.dataset_dict:Pushing split validation to the     ]8;id=701555;file:///home/jlortiz/data/anaconda3/envs/py310/lib/python3.10/site-packages/datasets/dataset_dict.py\dataset_dict.py]8;;\:]8;id=60185;file:///home/jlortiz/data/anaconda3/envs/py310/lib/python3.10/site-packages/datasets/dataset_dict.py#1532\1532]8;;\
                             Hub.                                                                                  

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
hf_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ng_tags'],
        num_rows: 2090
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ng_tags'],
        num_rows: 523
    })
})